In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()

In [2]:
spark

In [14]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import RFormula
import matplotlib.pyplot as plt
import numpy as np
import datetime

## Import natural language processing toolkit 
import re
import nltk
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pyspark.sql.functions import udf


In [4]:
path = "s3://bailey-bucket-dtb/user_dedup.json"
reviews = spark.read.json(path)

In [8]:
reviews.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [23]:
reviews.groupBy('overall').agg(count('reviewerID').alias('count')).sort('overall').show()

+-------+--------+
|overall|   count|
+-------+--------+
|    1.0| 6712117|
|    2.0| 4265230|
|    3.0| 7049302|
|    4.0|15480820|
|    5.0|49169670|
+-------+--------+



In [24]:
sample_rev = reviews.sample(False, .01, 12345)

In [30]:
# remove punctuation
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    return nopunct

# binarize rating
def convert_rating(rating):
    rating = int(rating)
    if rating >=4: return 1
    else: return 0

# udf
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))

# apply to review raw data
review_df = sample_rev.select('asin', 'helpful', 'overall', 'reviewerID', punct_remover('reviewText'), 'reviewTime', 'summary', 'unixReviewTime') #, rating_convert('overall')

# review_df = review_df.withColumnRenamed('<lambda>(reviewText)', 'reviewText')\
#                      .withColumn('label', review_df["<lambda>(overall)"].cast(IntegerType()))\
#                      .drop('<lambda>(overall)')\
#                      .limit(1000000)

review_df.show(5)

+----------+-------+-------+--------------------+--------------------+-----------+--------------------+--------------+
|      asin|helpful|overall|          reviewerID|<lambda>(reviewText)| reviewTime|             summary|unixReviewTime|
+----------+-------+-------+--------------------+--------------------+-----------+--------------------+--------------+
|B003ESE4TI| [0, 0]|    5.0|A000063614T1OE0BU...|I buy these for m...| 04 5, 2013|         Great value|    1365120000|
|B004GWQBWY| [0, 0]|    5.0|A00009661LC9LQPGK...|Loftek is an awes...|10 26, 2012|      Awsome Camera!|    1351209600|
|1479294608| [1, 1]|    5.0|A00009921ASVLX5LO...|This was a great ...|03 23, 2014|         great book!|    1395532800|
|B003EO1H7E| [0, 0]|    5.0|A000187635I595IAV...|I am a newbie at ...|01 26, 2013|Was asked for my ...|    1359158400|
|B00I8NZWWW| [0, 1]|    5.0|A000387412WV6NRZN...|I have just recei...| 07 1, 2014|They are absolute...|    1404172800|
+----------+-------+-------+--------------------

In [31]:
review_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- <lambda>(reviewText): string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [32]:
review_df.groupBy('overall').agg(count('reviewerID').alias('count')).sort('overall').show()

+-------+------+
|overall| count|
+-------+------+
|    1.0| 67031|
|    2.0| 42286|
|    3.0| 70503|
|    4.0|153976|
|    5.0|492050|
+-------+------+



In [34]:
# tokenize
tok = Tokenizer(inputCol="reviewText", outputCol="words")
review_tokenized = tok.transform(review_df)

# remove stop words
stopword_rm = StopWordsRemover(inputCol='words', outputCol='words_nsw')
review_tokenized = stopword_rm.transform(review_tokenized)

review_tokenized.show(5)

NameError: name 'Tokenizer' is not defined

In [15]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

test_pdf = sample_rev.select("*").toPandas()

/home/hadoop/miniconda/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [33]:
# from textblob import TextBlob

# def preprocess(ReviewText):
#     ReviewText = ReviewText.str.replace("(<br/>)", "")
#     ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
#     ReviewText = ReviewText.str.replace('(&amp)', '')
#     ReviewText = ReviewText.str.replace('(&gt)', '')
#     ReviewText = ReviewText.str.replace('(&lt)', '')
#     ReviewText = ReviewText.str.replace('(\xa0)', ' ')  
#     return ReviewText
# test_pdf['reviewText'] = preprocess(test_pdf['reviewText'])

# test_pdf['polarity'] = test_pdf['reviewText'].map(lambda text: TextBlob(text).sentiment.polarity)
# test_pdf['review_len'] = test_pdf['reviewText'].astype(str).apply(len)
# test_pdf['word_count'] = test_pdf['reviewText'].apply(lambda x: len(str(x).split()))